In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn.metrics import roc_curve, confusion_matrix, auc
from imblearn.over_sampling import SMOTE

ModuleNotFoundError: No module named 'sklearn.datasets'

Import scikit libraries

In [47]:
# --- Model selection utilities ---
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

# --- Preprocessing tools ---
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

# --- Classification models ---
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import RocCurveDisplay
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
# --- Evaluation metrics ---
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve,
    confusion_matrix,
    classification_report,
    auc,
)

from sklearn.inspection import permutation_importance

ModuleNotFoundError: No module named 'sklearn.inspection'

# 1. Data Loading



In [48]:
df = pd.read_csv('../../data/diabetes.csv')
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# 2. Train-Test Split (80/20)

In [46]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 3. Class Balancing (on Training Set Only)
Use SMOTE or ADASYN to upsample minority class (Outcome = 1):

In [29]:
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)

NameError: name 'SMOTE' is not defined

# 4. Nested Cross validation for model selection and hyperparameter tuning

In [ ]:
param_grid = {
    'knn__n_neighbors': [3, 5, 7, 9],
    'knn__metric': ['euclidean', 'manhattan']
}

# 5. Evaluation Metrics and Visualization

In [ ]:
# ROC Curve per outer fold
# Final ROC for best model
# F1-score and confusion matrix on test set
# Save best model for interpretation

# 6. Permuation feature importance

In [ ]:
perm_result = permutation_importance(
    best_model, X_test, y_test, scoring='f1', n_repeats=30, random_state=42
)

# 7. Decision Threshold & Interpretation

In [ ]:
prob = best_model.predict_proba(X_test)[:, 1]
plt.scatter(X_test["Glucose"], prob, c=y_test, cmap="bwr")
plt.axhline(0.5, linestyle="--", color="gray")
